# __Chapter 0. Preparing data__

- In this chapter, we are going to download the '__adjusted closing price__' of stocks listed in the stock index __Russel 1000__. 
- The reason that we use Russel 1000 instead of S&P 500 is that the number of stocks left is not many when we filter stocks with missing values from 1999 to 2019, if we use S&P 500.
- We are aware of that filtering stocks with missing values will cause __Survivorship bias__. This will be fixed in the future research
- The adjusted closing price is going to turn into daily returns of stocks and then be used when we optimize a portfolio.

#### __Description of Data__
- __Timespan :__ Nov.1999 - Nov.2019 (recent 20 years)
- __Stock index used :__ Russel 1000
- __Source :__ Yahoo Finance
- __Library used for downloading data__ : Pandas datareader

#### __Contents__

- [__Step 01. Loading stock data__](#Step-01.-Loading-stock-data)
- [__Step 02. Filtering tickers__](#Step-02.-Filtering-tickers)
- [__Step 03. Creating a daily_price_df__](#Step-03.-Creating-a-daily_price_df)
- [__Step 04. Grouping daily_return by a year__](#Step-04.-Grouping-daily_return-by-a-year)
- [__Step 05. Downloading industry information__](#Step-05.-Downloading-industry-information)

---

## Step 01. Loading stock data

__1. Importing required libraries__

In [1]:
# for importing libraries again that is already imported, just in case that any change is made in libraries
%load_ext autoreload
%autoreload 2

# libraries for general work
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

# for downloading stock data
import pandas_datareader.data as pdr
import datetime

# for downloading industry information of each company
# source : https://github.com/davidastephens/pandas-finance
from pandas_finance import Equity

# Disable printing all warnings on python 
import warnings
warnings.filterwarnings("ignore")

__2. Loading Russel 1000 index constituents__
- loading the tickers consisting the Russel 1000 index from the beginning of __Nov.1999__ till the end of __Nov.2019__

In [2]:
russel1000_tickers = pd.read_csv("./data/russel1000_constituents.csv", header=None)
russel1000_tickers = list(russel1000_tickers.iloc[:,0])
russel1000_tickers[:5]

['TWOU', 'MMM', 'ABT', 'ABBV', 'ABMD']

In [3]:
len(russel1000_tickers)

968

__3. Setting dates of `start` and `end` (the start and end dates when we download stocks)__

In [4]:
# We will look at stock prices over the past year, starting at January 1, 2016
start = (1999, 11, 1)
start = datetime.datetime(*start)
end = (2019, 12, 1)
end = datetime.datetime(*end)

__4. Download stock data__

In [5]:
stock_data_dict = {}
missing_tickers = []

for ticker in tqdm(russel1000_tickers):
    try:
        stock_data = pdr.DataReader(ticker, 'yahoo', start, end)
        stock_data_dict[ticker] = stock_data
    except:
        print(f"The following ticker made an error : {ticker}")
        missing_tickers.append(ticker)

The following ticker made an error : BHGE
The following ticker made an error : BRK.B
The following ticker made an error : BFA
The following ticker made an error : BFB
The following ticker made an error : DPS
The following ticker made an error : DNB
The following ticker made an error : EGN
The following ticker made an error : FDC
The following ticker made an error : HEI.A
The following ticker made an error : HPT
The following ticker made an error : LEN.B
The following ticker made an error : LGF.A
The following ticker made an error : PF
The following ticker made an error : TSRO
The following ticker made an error : TMK
The following ticker made an error : TRCO
The following ticker made an error : DATA
The following ticker made an error : ULTI
The following ticker made an error : VVC
The following ticker made an error : VSM
The following ticker made an error : USG
The following ticker made an error : UBNT
The following ticker made an error : WFT
The following ticker made an error : JW.A
Th

In [6]:
len(stock_data_dict)

943

In [7]:
len(missing_tickers)

25

__5. Checking `missing_tickers` once again__

In [8]:
replaced_missing_tickers = []

for ticker in missing_tickers:
    replaced_ticker = ticker.replace('.','-')
    replaced_missing_tickers.append(replaced_ticker)

In [9]:
for ticker in tqdm(replaced_missing_tickers):
    try:
        stock_data = pdr.DataReader(ticker, 'yahoo', start, end)
        stock_data_dict[ticker] = stock_data
    except:
        print(f"The following ticker made an error : {ticker}")
        missing_tickers.append(ticker)

The following ticker made an error : BHGE
The following ticker made an error : BFA
The following ticker made an error : BFB
The following ticker made an error : DPS
The following ticker made an error : DNB
The following ticker made an error : EGN
The following ticker made an error : FDC
The following ticker made an error : HPT
The following ticker made an error : PF
The following ticker made an error : TSRO
The following ticker made an error : TMK
The following ticker made an error : TRCO
The following ticker made an error : DATA
The following ticker made an error : ULTI
The following ticker made an error : VVC
The following ticker made an error : VSM
The following ticker made an error : USG
The following ticker made an error : UBNT
The following ticker made an error : WFT
The following ticker made an error : WP



In [10]:
len(stock_data_dict)

948

In [11]:
# Saving
with open('data/russel1000_raw_data_dict.pickle', 'wb') as f: # 
    pickle.dump(stock_data_dict, f, pickle.HIGHEST_PROTOCOL)

---

## Step 02. Filtering tickers

__1. Filtering tickers with missing value__

In [12]:
ticker_list_in_stock_data_dict = list(stock_data_dict.keys())
stock_data_dict_len_list = []

for ticker in ticker_list_in_stock_data_dict:
    stock_data_dict_len_list.append(len(stock_data_dict[ticker]))

In [13]:
maximum_len = pd.Series(stock_data_dict_len_list).max()
maximum_len

5053

In [14]:
filtered_stock_data_dict = {}

for ticker in ticker_list_in_stock_data_dict:
    if len(stock_data_dict[ticker]) == maximum_len:
        filtered_stock_data_dict[ticker] = stock_data_dict[ticker]

In [15]:
len(filtered_stock_data_dict)

590

In [16]:
# # Saving
# with open('data/russel1000_stock_data_dict.pickle', 'wb') as f: # 
#     pickle.dump(filtered_stock_data_dict, f, pickle.HIGHEST_PROTOCOL)

In [17]:
# Loading
with open('data/russel1000_stock_data_dict.pickle', 'rb') as f:
    filtered_stock_data_dict = pickle.load(f)

---

## Step 03. Creating a daily_price_df

__1. Creating a dataframe of price : `daily_price_df`__

In [18]:
daily_price_dict = {}
filtered_tickers = list(filtered_stock_data_dict.keys())

for ticker in filtered_tickers:
    daily_price_dict[ticker] = filtered_stock_data_dict[ticker].loc[:,'Adj Close']

In [19]:
daily_price_df = pd.DataFrame.from_dict(daily_price_dict)
del daily_price_df.index.name

daily_price_df.head(3)

,MMM,ABT,ABMD,ACHC,ATVI,ADBE,AMD,AES,AMG,AFL,...,ZBRA,ZION,XEL,XRX,XLNX,YUM,BRK-B,HEI-A,LGF-A,JW-A
1999-11-01,26.186508,7.625059,10.2500,3.750,1.057096,17.013006,10.15625,22.383265,17.685398,5.895580,...,26.222221,44.172218,8.828288,59.627827,28.284966,4.807233,41.740002,1.899360,1.423901,12.179971
1999-11-02,26.430267,7.187653,10.3125,3.750,1.071191,16.888933,10.28125,22.780302,17.523146,6.068317,...,25.388889,45.481014,8.853581,57.188522,28.801279,4.733161,42.820000,1.891793,1.385417,12.134530
1999-11-03,26.325787,6.963037,10.2500,3.875,1.089984,17.075039,10.65625,22.209560,17.604273,6.150932,...,24.944445,44.920109,8.727110,57.324043,30.417568,4.651680,43.060001,1.755583,1.539352,11.725501


__2. Checking the dtypes of__ `daily_price_df`
- Need to check the datatypes of data included in dataframe because some of the numbers may not be numbers(floats, ints, etc) but actually strings.
- If some strings are mixed with numbers in the dataframe, some computations such as `.cov()` cannot be performed.

In [20]:
daily_price_df.dtypes.value_counts()

float64    590
dtype: int64

- The datatype `object` in pandas means string, while `float64` means float numbere.
- Source : <a href="https://pbpython.com/pandas_dtypes.html">_Overview of Pandas Data Types_</a>

In [22]:
# # Saving
# with open('data/russel1000_daily_price_df.pickle', 'wb') as f: # 
#     pickle.dump(daily_price_df, f, pickle.HIGHEST_PROTOCOL)

In [23]:
# Loading
with open('data/russel1000_daily_price_df.pickle', 'rb') as f:
    daily_price_df = pickle.load(f)

---

## Step 04. Grouping daily_return by a year

__1. Creating a dataframe of return : `daily_return_df`__

In [24]:
daily_return_df = daily_price_df.pct_change()
daily_return_df = daily_return_df.dropna(axis=0)
daily_return_df.head(3)

,MMM,ABT,ABMD,ACHC,ATVI,ADBE,AMD,AES,AMG,AFL,...,ZBRA,ZION,XEL,XRX,XLNX,YUM,BRK-B,HEI-A,LGF-A,JW-A
1999-11-02,0.009309,-0.057364,0.006098,0.000000,0.013333,-0.007293,0.012308,0.017738,-0.009174,0.029299,...,-0.031780,0.029629,0.002865,-0.040909,0.018254,-0.015408,0.025874,-0.003984,-0.027027,-0.003731
1999-11-03,-0.003953,-0.031250,-0.006061,0.033333,0.017544,0.011019,0.036474,-0.025054,0.004630,0.013614,...,-0.017505,-0.012333,-0.014285,0.002370,0.056119,-0.017215,0.005605,-0.072000,0.111111,-0.033708
1999-11-04,0.002646,0.028863,0.021341,0.000000,0.008621,0.000908,-0.032258,-0.003352,0.006913,0.018315,...,-0.020045,0.009365,0.002898,0.011820,0.012546,0.000000,0.004645,0.000000,0.250000,0.019380


In [25]:
daily_return_df.tail(3)

,MMM,ABT,ABMD,ACHC,ATVI,ADBE,AMD,AES,AMG,AFL,...,ZBRA,ZION,XEL,XRX,XLNX,YUM,BRK-B,HEI-A,LGF-A,JW-A
2019-11-26,-0.005933,0.011007,0.035667,-0.017078,0.008189,0.008582,-0.020106,-0.001597,-0.010399,0.007919,...,-0.004981,-0.010165,0.006250,-0.011868,-0.006317,0.017753,-0.000730,0.004159,0.025082,0.010036
2019-11-27,0.008155,0.000000,0.013024,0.011480,0.005723,0.003767,0.010772,0.007467,0.003503,0.001644,...,0.030319,0.008256,0.006211,0.013055,0.023126,0.008922,0.007309,0.007374,0.018085,0.009725
2019-11-29,-0.004923,0.000351,-0.003100,-0.013497,0.006424,0.001521,-0.006597,0.001059,-0.006748,0.000365,...,-0.016731,-0.005792,-0.001137,0.003350,-0.006106,0.000298,-0.000907,0.007119,-0.022988,-0.010469


__2. Grouping price data by a year__ : `grouped_daily_price_df_list`

In [26]:
# past 252 days' daily returns are considered to calculate the covariance
seq_length = 252

# portfolio rebalnacing period is 60 days
rebalancing_period = 60
    
# sliced dataframes are stored in the list below temporarily
sliced_daily_return_df_list = []

# for validation
for i in range(0, (daily_return_df.shape[0]-(seq_length)+1), rebalancing_period):  # i gets bigger by 20 : 0, 20, 40,...
    sliced_daily_return_df = (daily_return_df.iloc[(i):(i+seq_length),:])
    sliced_daily_return_df_list.append(sliced_daily_return_df) # this is used for portfolio optimization, so can stay as pd.DataFrame

In [27]:
len(sliced_daily_return_df_list)

81

In [28]:
validation_daily_price_df_list = []
test_daily_price_df_list = []

# for validation
for i in range(len(sliced_daily_return_df_list)):
    if i <= (47-1):
        validation_daily_price_df_list.append(sliced_daily_return_df_list[i])
    elif i >= (47+4-1) : 
        test_daily_price_df_list.append(sliced_daily_return_df_list[i])

In [29]:
print(f"len(validation_daily_price_df_list) : {len(validation_daily_price_df_list)}")
print(f"len(test_daily_price_df_list) : {len(test_daily_price_df_list)}")

len(validation_daily_price_df_list) : 47
len(test_daily_price_df_list) : 31


In [32]:
# # Saving - validation data
# with open('./data/russel1000_daily_return_df_validation.pickle', 'wb') as f: 
#     pickle.dump(validation_daily_price_df_list, f, pickle.HIGHEST_PROTOCOL)

In [34]:
# # Saving - test data
# with open('./data/russel1000_daily_return_df_test.pickle', 'wb') as f: 
#     pickle.dump(test_daily_price_df_list, f, pickle.HIGHEST_PROTOCOL)

---

## Step 05. Downloading industry information

In [35]:
sector_dict = {}

sector_dict['Industrials'] = []
sector_dict['Healthcare'] = []
sector_dict['Communication Services'] = []
sector_dict['Technology'] = []
sector_dict['Utilities'] = []
sector_dict['Financial Services'] = []
sector_dict['Basic Materials'] = []
sector_dict['Real Estate'] = []
sector_dict['Consumer Defensive'] = []
sector_dict['Consumer Cyclical'] = []
sector_dict['Energy'] = []

In [36]:
for ticker in tqdm(filtered_tickers):
    try: 
        ticker_sector = Equity(ticker).sector
        sector_dict[ticker_sector].append(ticker)
    except:
        print(f"following ticker made an issue : {ticker}")

following ticker made an issue : BBT
following ticker made an issue : CHK
following ticker made an issue : GD
following ticker made an issue : JEC
following ticker made an issue : STI



In [37]:
sector_dict["Financial Services"].append('BBT')
sector_dict["Energy"].append('CHK')
sector_dict["Industrials"].append('GD')
sector_dict["Industrials"].append('JEC')
sector_dict["Financial Services"].append('STI')

In [38]:
industry_sum = 0

for industry in sector_dict.keys():
    print(f"{industry} : {len(sector_dict[industry])}")
    industry_sum+= int(len(sector_dict[industry]))

print()
print(f"Total number of tickers : {industry_sum}")

Industrials : 93
Healthcare : 68
Communication Services : 18
Technology : 67
Utilities : 31
Financial Services : 92
Basic Materials : 30
Real Estate : 49
Consumer Defensive : 37
Consumer Cyclical : 73
Energy : 32

Total number of tickers : 590


In [39]:
# Saving
with open('./data/russel1000_sector_dict.pickle', 'wb') as f: # 
    pickle.dump(sector_dict, f, pickle.HIGHEST_PROTOCOL)